In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

from keras import layers, models, backend, losses
from keras.models import Model
from keras.callbacks import EarlyStopping

In [ ]:
print(tf.__version__)

In [2]:
# Train and Test Image Directories
train_img_dir = "D:Development/Data_Science_Datasets/Monkeypox_skinlesion/RestNest50/Train/"
test_img_dir = "D:Development/Data_Science_Datasets/Monkeypox_skinlesion/RestNest50/Test/"

# Basic variables
val_batch_size = 51
val_img_height = 224
val_img_width = 224
val_validation_split = None
val_subset = None
val_shuffle = True
val_labels = 'inferred'
val_seed = 56
val_channels = 3
val_interpolation = 'gaussian'

In [3]:
train_img_ds = tf.keras.utils.image_dataset_from_directory(
    train_img_dir,
    validation_split = val_validation_split,
    subset = val_subset,
    seed = val_seed,
    image_size=(val_img_height, val_img_width),
    batch_size = val_batch_size, 
    shuffle = val_shuffle,
    labels = val_labels,
    interpolation = val_interpolation)

Found 2156 files belonging to 2 classes.


In [4]:
print(train_img_ds)

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [5]:
train_img_df = None
train_label_df = None

for image_data, labels_data in train_img_ds:
    print(image_data.shape)
    print(labels_data.shape)
    break    

(51, 224, 224, 3)
(51,)


In [6]:
for image_data, labels_data in train_img_ds:
    print(image_data.numpy())
    print(labels_data.numpy())
    break

[[[[ 36.747383  51.508972  58.628174]
   [ 32.825073  45.71856   53.825066]
   [ 26.545227  36.43872   46.43872 ]
   ...
   [ 32.43872   19.931578  42.131893]
   [ 31.1192    19.306824  43.106503]
   [ 29.371815  19.13341   43.133408]]

  [[ 38.523186  53.284775  60.40398 ]
   [ 34.083817  46.977306  55.07247 ]
   [ 28.330862  38.235695  48.129192]
   ...
   [ 34.140537  20.38065   43.201664]
   [ 32.82101   20.821016  44.703167]
   [ 30.639036  20.21301   44.213013]]

  [[ 41.510487  56.27208   63.391285]
   [ 37.        49.882145  57.893486]
   [ 31.319519  41.19032   50.30817 ]
   ...
   [ 36.39199   22.106506  45.106506]
   [ 35.083817  22.714508  46.691814]
   [ 34.038086  22.200317  46.213005]]

  ...

  [[ 65.83001   26.786982  47.868095]
   [ 63.034023  25.09516   46.011337]
   [ 59.999992  22.4985    43.29683 ]
   ...
   [199.51283  219.51282  210.78697 ]
   [201.85944  218.96594  211.      ]
   [203.56123  218.79965  211.68047 ]]

  [[ 72.02873   30.642387  52.442066]
   [ 68

In [7]:
test_img_ds = tf.keras.utils.image_dataset_from_directory(
    test_img_dir,
    validation_split = val_validation_split,
    subset = val_subset,
    seed = val_seed,
    image_size=(val_img_height, val_img_width),
    batch_size = val_batch_size, 
    shuffle = val_shuffle,
    labels = val_labels,
    interpolation = val_interpolation)

Found 45 files belonging to 2 classes.


In [8]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [9]:
norm_train_img_ds = train_img_ds.map(lambda x, y: (normalization_layer(x), y))

In [10]:
class_names = train_img_ds.class_names

In [11]:
norm_test_img_ds = test_img_ds.map(lambda x, y: (normalization_layer(x), y))

In [12]:
for image_data, labels_data in norm_train_img_ds:
    print(image_data.numpy())
    print(labels_data.numpy())
    
    print(image_data.shape)
    print(labels_data.shape)
    
    break

[[[[0.0000000e+00 0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00]
   ...
   [0.0000000e+00 0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00]]

  [[0.0000000e+00 0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00]
   ...
   [0.0000000e+00 0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00]]

  [[0.0000000e+00 0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00]
   ...
   [0.0000000e+00 0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00]]

  ...

  [[0.0000000e+00 0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.000000

In [13]:


model = tf.keras.applications.resnet50.ResNet50(include_top=True,
    weights='imagenet')

In [14]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                  'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block1_add[0][0]']       
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 7, 7, 512)    1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv5_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_1_relu (Activatio  (None, 7, 7, 512)   0           ['conv5_block2_1_bn[0][0]']      
 n)       

In [15]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

backend.set_value(model.optimizer.learning_rate, 0.0001)

history = model.fit(norm_train_img_ds, epochs=15, 
                    validation_data=(norm_test_img_ds))

Epoch 1/15
43/43 [==============================] - 833s 19s/step - loss: 2.2417 - accuracy: 0.6818 - val_loss: 2.5361 - val_accuracy: 0.5556
Epoch 2/15
43/43 [==============================] - 824s 19s/step - loss: 0.0657 - accuracy: 0.9814 - val_loss: 3.5387 - val_accuracy: 0.5556
Epoch 3/15
43/43 [==============================] - 878s 20s/step - loss: 0.0172 - accuracy: 0.9972 - val_loss: 3.6047 - val_accuracy: 0.5556
Epoch 4/15
43/43 [==============================] - 836s 19s/step - loss: 0.0096 - accuracy: 0.9972 - val_loss: 3.9601 - val_accuracy: 0.5556
Epoch 5/15
43/43 [==============================] - 852s 20s/step - loss: 0.0288 - accuracy: 0.9921 - val_loss: 3.8402 - val_accuracy: 0.3556
Epoch 6/15
43/43 [==============================] - 870s 20s/step - loss: 0.0157 - accuracy: 0.9968 - val_loss: 3.9748 - val_accuracy: 0.0000e+00
Epoch 7/15
43/43 [==============================] - 884s 21s/step - loss: 0.0093 - accuracy: 0.9977 - val_loss: 3.7981 - val_accuracy: 0.0000e+0

In [1]:
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label = 'Val-Accuracy')
plt.plot(history.history['loss'], label = 'Loss')
plt.plot(history.history['val_loss'], label = 'Val-Loss')
plt.xlabel('Epoch')
plt.ylabel('Metrics')
plt.ylim([0.0, 1.1])
plt.legend(loc='lower right')
plt.title('ResNet50 Learning-rate 1e-4')

NameError: name 'plt' is not defined

In [20]:
val_img_dir = "D:Development/Data_Science_Datasets/Monkeypox_skinlesion/Val/"

val_img_ds = tf.keras.utils.image_dataset_from_directory(
    val_img_dir,
    validation_split = val_validation_split,
    subset = val_subset,
    seed = val_seed,
    image_size=(val_img_height, val_img_width),
    batch_size = val_batch_size, 
    shuffle = val_shuffle,
    labels = val_labels,
    interpolation = val_interpolation)

Found 406 files belonging to 2 classes.


In [21]:
test_loss, test_acc = model.evaluate(val_img_ds,  verbose=2)
print(test_acc)

8/8 - 34s - loss: 30.4660 - accuracy: 0.5739 - 34s/epoch - 4s/step
0.5738916397094727


In [22]:
norm_val_img_ds = val_img_ds.map(lambda x, y: (normalization_layer(x), y))

In [23]:
test_loss, test_acc = model.evaluate(norm_val_img_ds,  verbose=2)
print(test_acc)

8/8 - 33s - loss: 2.6717 - accuracy: 0.5468 - 33s/epoch - 4s/step
0.546798050403595
